<a href="https://colab.research.google.com/github/AdrianGuel/PhDThesis/blob/main/MPC/MPConpythonunfinished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install libraries
!pip install torchsde
!pip install KDEpy
!pip install casadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 5.0 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 193 kB 58.9 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 420 kB 6.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.2 MB 9

In [ ]:
def shift():
  st = x0;
  con = u(1,:)'
  #RK simulation
  k1 = f(st, con);  
  k2 = f(st + T/2*k1, con)
  k3 = f(st + T/2*k2, con)
  k4 = f(st + T*k3, con) 
  st=st +T/6*(k1 +2*k2 +2*k3 +k4)
  #EulerSimu
  %f_value = f(st,con)
  %st = st+ (T*f_value)
  x0 = full(st)
  t0 = t0 + T
  u0 = [u(2:size(u,1),:);u(size(u,1),:)]
  return t0, x0, u0

In [ ]:
from casadi import *
y0=5/6+2;
b0=30;
bf=3;
yf=1/30+2;
tf=0.3;

h = 1e-3; %[s]
%h = 1e-2; %[s]
N = 5; % prediction horizon
%N = 20; % prediction horizon
gamma=1;

D_max = inf; D_min = 0;
%u_max = -gamma; u_min = -inf;
u_max = inf; u_min = -inf;

x = SX.sym('x'); Sigma = SX.sym('Sigma'); time = SX.sym('time');
dx = SX.sym('dx'); dSigma = SX.sym('dSigma');

states = [x; Sigma]; n_states = length(states);
u = SX.sym('u'); D0=SX.sym('D0');
controls = [u;D0]; n_controls = length(controls);

rhs = [-gamma*x^3+u-0.5*gamma*6*x*Sigma;-6*gamma*Sigma*(x^2)+2*D0];
%rhs = [(gamma+u)*x;2*gamma*Sigma+2*D0];
f = Function('f',{states,controls},{rhs}); % nonlinear mapping function f(x,u)

Erhs = sqrt(((-gamma*x^3+u-0.5*gamma*6*x*Sigma)^2)/Sigma+0.5*((-6*gamma*Sigma*(x^2)+2*D0)/Sigma)^2);
%Erhs = (((gamma+u)*x)^2)/Sigma+0.5*((2*gamma*Sigma+2*D0)/Sigma)^2;
E = Function('E',{states,controls},{Erhs});


U = SX.sym('U',n_controls,N); % Decision variables (controls)
P = SX.sym('P',n_states + n_states);
% parameters (which include the initial state and th        %obj = obj+(st-P(3:4))'*Q*(st-P(3:4)) + con'*R*con; % calculate obje reference state)

X = SX.sym('X',n_states,(N+1));
% A vector that repr0;  %esents the states over the optimization problem.

obj = 0; % Objective function
g = [];  % constraints vector1./sqrt(1./(2*xx(2,1)))

Q = zeros(n_states,n_states);  Q(1,1) = 1e2; Q(2,2) = 8e2; % weighing matrices (states)
R = zeros(n_controls,n_controls);  R(1,1) = 1e-5; R(2,2) =1e-5; % weighing matrices (control)
I = 1e3;

st  = X(:,1); % initial state
g = [g;st-P(1:2)]; % initial condition constraints
for k = 1:N
        %if k==1
        st = X(:,k);  con = U(:,k);

        Et = E(st, con);   % new % 151.7078 %558.4475 %50.1153
        obj = obj+I*(Et-50.1153)^2+(st-P(3:4))'*Q*(st-P(3:4))+ con'*R*con; % calculate obj
        
        st_next = X(:,k+1);
        k1 = f(st, con);   % new 
        k2 = f(st + h/2*k1, con); % new
        k3 = f(st + h/2*k2, con); % new
        k4 = f(st + h*k3, con); % new
        st_next_RK4=st +h/6*(k1 +2*k2 +2*k3 +k4); % new    
        % f_value = f(st,con);1./sqrt(1./(2*xx(2,1)))
        % st_next_euler = st+ (h*f_value);
        % g = [g;st_next-st_next_euler]; % compute constraints
        g = [g;st_next-st_next_RK4]; % compute constraints % new
end
%obj=sqrt(obj/N);
% make the decision variable one column  vector
OPT_variables = [reshape(X,2*(N+1),1);reshape(U,2*N,1)];

nlp_prob = struct('f', obj, 'x', OPT_variables, 'g', g, 'p', P);

opts = struct;
opts.ipopt.max_iter = 2000;
opts.ipopt.print_level =0;%0,3
opts.print_time = 0;
opts.ipopt.acceptable_tol =1e-8;
opts.ipopt.acceptable_obj_change_tol = 1e-6;

solver = nlpsol('solver', 'ipopt', nlp_prob,opts);

args = struct;

args.lbg(1:2*(N+1)) =  0; %-1e-20  % Equality constraints
args.ubg(1:2*(N+1)) = 0; %1e-20   % Equality constraints

args.lbx(1:2:2*(N+1),1) = -6; %state x lower bound
args.ubx(1:2:2*(N+1),1) = 6; %state x upper bound
args.lbx(2:2:2*(N+1),1) = -4; %state Sigma lower bound
args.ubx(2:2:2*(N+1),1) = 4; %state Sigma upper bound
% args.lbx(3:4:4*(N+1),1) = -4; %state Sigma lower bound
% args.ubx(3:4:4*(N+1),1) = 4; %state Sigma upper bound
% args.lbx(4:4:4*(N+1),1) = -2; %state dSigma lower bound
% args.ubx(4:4:4*(N+1),1) = 2; %state dSigma upper bound

args.lbx(2*(N+1)+1:2:2*(N+1)+2*N,1) = u_min; %v lower bound
args.ubx(2*(N+1)+1:2:2*(N+1)+2*N,1) = u_max; %v upper bound
args.lbx(2*(N+1)+2:2:2*(N+1)+2*N,1) = D_min; %omega lower bound
args.ubx(2*(N+1)+2:2:2*(N+1)+2*N,1) = D_max; %omega upper bound
%----------------------------------------------
% ALL OF THE ABOVE IS JUST A PROBLEM SET UP


% THE SIMULATION LOOP SHOULD START FROM HERE
%-------------------------------------------
t0 = 0;
%x0 = [0.5;(1/(2*0.03))];
%xs = [1/30;(1/(2*0.3))]; 
% x0 = [5/6;(1/(2*0.3))];    % initial condition.
% xs = [1/30;(1/(2*3))]; % Reference posture.
%Geodesic II
x0 = [y0;(1/(2*b0))];
xs = [yf;(1/(2*bf))]; 

xx(:,1) = x0; % xx contains the history of states
t(1) = t0;

u0 = zeros(N,2);        % two control inputs for each robot
X0 = repmat(x0,1,N+1)'; % initialization of the states decision variables
u0(:,2)=(1/(2*30));

sim_tim = tf; % Maximum simulation time

% Start MPC
mpciter = 0;
xx1 = [];
u_cl=[];

% the main simulaton loop... it works as long as the error is greater
% than 10^-6 and the number of mpc steps is less than its maximum
% value.
main_loop = tic;
while ((norm((x0-xs),2) > 1e-3) && (mpciter < sim_tim / h))
    args.p   = [x0;xs]; % set the values of the parameters vector
    % initial value of the optimization variables
    args.x0  = [reshape(X0',2*(N+1),1);reshape(u0',2*N,1)];
    sol = solver('x0', args.x0, 'lbx', args.lbx, 'ubx', args.ubx,...
        'lbg', args.lbg, 'ubg', args.ubg,'p',args.p);
    u = reshape(full(sol.x(2*(N+1)+1:end))',2,N)'; % get controls only from the solution
    xx1(:,1:2,mpciter+1)= reshape(full(sol.x(1:2*(N+1)))',2,N+1)'; % get solution TRAJECTORY
    u_cl= [u_cl ; u(1,:)];
    t(mpciter+1) = t0;
    % Apply the control and shift the solution
    [t0, x0, u0] = shift(h, t0, x0, u,f);
    xx(:,mpciter+2) = x0;
    X0 = reshape(full(sol.x(1:2*(N+1)))',2,N+1)'; % get solution TRAJECTORY
    % Shift trajectory to initialize the next step
    X0 = [X0(2:end,:);X0(end,:)];
    %mpciter
    mpciter = mpciter + 1;
end
main_loop_time = toc(main_loop);
ss_error = norm((x0-xs),2)
average_mpc_time = main_loop_time/(mpciter+1)
u_cl=u_cl';
Ex=sqrt(((-gamma.*xx(1,1:end-1).^3+u_cl(1,:)-0.5*gamma*6.*xx(1,1:end-1).*xx(2,1:end-1)).^2)./xx(2,1:end-1)+0.5*((-6*gamma.*xx(2,1:end-1).*(xx(1,1:end-1).^2)+2*u_cl(2,:))./xx(2,1:end-1)).^2);
ILx=cumtrapz(t,Ex);
